# concat  oof df


In [1]:
import os
import gc 

import numpy as np
import pandas as pd
from pandarallel import pandarallel
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


In [2]:
output_dir = os.path.join("/kaggle", "working", "_oof")
# exp_name = "exp006_addlayer"
exp_name = "exp008_meanstd_nonavepool"
print("load oof df")
oof_df = pd.DataFrame()
for i in range(5):
    print("fold", i)
    df = pd.read_parquet(os.path.join(output_dir, exp_name, f"oof_df_fold{i}.parquet"))
    for col in df.columns:
        # 64bit float -> 16bit float
        if df[col].dtype == np.float64:
            df[col] = df[col].astype(np.float16)
        # 64bit int -> 16bit int
        elif df[col].dtype == np.int64:
            df[col] = df[col].astype(np.int16)

    oof_df = pd.concat([oof_df, df], axis=0)

load oof df
fold 0


fold 1
fold 2
fold 3
fold 4


In [3]:
oof_df.columns

Index(['series_id', 'step', 'timestamp', 'anglez', 'enmo', 'anglez_absdiff',
       'enmo_absdiff', 'anglez_absdiff_ave', 'enmo_absdiff_ave', 'event',
       'event_onset', 'event_wakeup', 'date', 'series_date_key',
       'series_date_key_str', 'fold', 'class_pred', 'class_target',
       'event_pred'],
      dtype='object')

In [4]:
oof_df

,series_id,step,timestamp,anglez,enmo,anglez_absdiff,enmo_absdiff,anglez_absdiff_ave,enmo_absdiff_ave,event,event_onset,event_wakeup,date,series_date_key,series_date_key_str,fold,class_pred,class_target,event_pred
6644251,0ec9fc461819,0,2019-03-27T14:00:00-0400,-11.080400,0.0335,0.000000,0.0000,0.0,0.0,0.0,0,0,2019-03-27,400,0ec9fc461819_2019-03-27,0,0.053497,0.0,0.051514
6644252,0ec9fc461819,1,2019-03-27T14:00:05-0400,-16.320299,0.0562,5.239899,0.0227,0.0,0.0,0.0,0,0,2019-03-27,400,0ec9fc461819_2019-03-27,0,0.014214,0.0,0.000000
6644253,0ec9fc461819,2,2019-03-27T14:00:10-0400,-18.830999,0.0441,2.510700,0.0121,0.0,0.0,0.0,0,0,2019-03-27,400,0ec9fc461819_2019-03-27,0,0.007229,0.0,0.000000
6644254,0ec9fc461819,3,2019-03-27T14:00:15-0400,-18.533800,0.0408,0.297199,0.0033,0.0,0.0,0.0,0,0,2019-03-27,400,0ec9fc461819_2019-03-27,0,0.006878,0.0,0.000000
6644255,0ec9fc461819,4,2019-03-27T14:00:20-0400,-11.525700,0.0520,7.008101,0.0112,0.0,0.0,0.0,0,0,2019-03-27,400,0ec9fc461819_2019-03-27,0,0.005859,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124991578,f981a0805fd0,405715,2019-01-26T22:59:35-0500,59.337601,0.0000,0.012897,0.0000,0.0,0.0,0.0,0,0,2019-01-26,7524,f981a0805fd0_2019-01-26,4,0.737305,0.0,-0.000000
124991579,f981a0805fd0,405716,2019-01-26T22:59:40-0500,59.342098,0.0000,0.004498,0.0000,0.0,0.0,0.0,0,0,2019-01-26,7524,f981a0805fd0_2019-01-26,4,0.740723,0.0,-0.000000
124991580,f981a0805fd0,405717,2019-01-26T22:59:45-0500,59.359200,0.0000,0.017101,0.0000,0.0,0.0,0.0,0,0,2019-01-26,7524,f981a0805fd0_2019-01-26,4,0.740723,0.0,-0.000000
124991581,f981a0805fd0,405718,2019-01-26T22:59:50-0500,59.382702,0.0000,0.023502,0.0000,0.0,0.0,0.0,0,0,2019-01-26,7524,f981a0805fd0_2019-01-26,4,0.739746,0.0,-0.000000


In [5]:
oof_df = oof_df.reset_index(drop=True)

In [6]:
oof_df = oof_df.drop(["timestamp", "anglez_absdiff", "enmo_absdiff", "anglez_absdiff_ave", "enmo_absdiff_ave", "class_target", "event_pred"], axis=1)

In [7]:
oof_df

,series_id,step,anglez,enmo,event,event_onset,event_wakeup,date,series_date_key,series_date_key_str,fold,class_pred
0,0ec9fc461819,0,-11.080400,0.0335,0.0,0,0,2019-03-27,400,0ec9fc461819_2019-03-27,0,0.053497
1,0ec9fc461819,1,-16.320299,0.0562,0.0,0,0,2019-03-27,400,0ec9fc461819_2019-03-27,0,0.014214
2,0ec9fc461819,2,-18.830999,0.0441,0.0,0,0,2019-03-27,400,0ec9fc461819_2019-03-27,0,0.007229
3,0ec9fc461819,3,-18.533800,0.0408,0.0,0,0,2019-03-27,400,0ec9fc461819_2019-03-27,0,0.006878
4,0ec9fc461819,4,-11.525700,0.0520,0.0,0,0,2019-03-27,400,0ec9fc461819_2019-03-27,0,0.005859
...,...,...,...,...,...,...,...,...,...,...,...,...
127950904,f981a0805fd0,405715,59.337601,0.0000,0.0,0,0,2019-01-26,7524,f981a0805fd0_2019-01-26,4,0.737305
127950905,f981a0805fd0,405716,59.342098,0.0000,0.0,0,0,2019-01-26,7524,f981a0805fd0_2019-01-26,4,0.740723
127950906,f981a0805fd0,405717,59.359200,0.0000,0.0,0,0,2019-01-26,7524,f981a0805fd0_2019-01-26,4,0.740723
127950907,f981a0805fd0,405718,59.382702,0.0000,0.0,0,0,2019-01-26,7524,f981a0805fd0_2019-01-26,4,0.739746


In [8]:
gc.collect()

50

In [9]:
for col in oof_df.columns:
    # 64bit float -> 16bit float
    if oof_df[col].dtype == np.float16:
        oof_df[col] = oof_df[col].astype(np.float64)
    # 64bit int -> 16bit int
    elif df[col].dtype == np.int16:
        df[col] = df[col].astype(np.int64)


In [10]:
oof_df.to_parquet(os.path.join(output_dir, exp_name, "oof_df.parquet"))

In [11]:
oof_df["event_onset"].value_counts()

0    127946116
1         4793
Name: event_onset, dtype: int64

In [12]:
key_onset = oof_df.groupby('series_date_key')['event_onset'].sum()
key_wakeup = oof_df.groupby('series_date_key')['event_wakeup'].sum()

In [13]:
key_event = pd.concat([key_onset, key_wakeup], axis=1)
key_event["event_count"] = key_event["event_onset"] + key_event["event_wakeup"]

In [14]:
secondstg_target_key = list(key_event[key_event["event_count"] != 0].index)
not_target_key = list(key_event[key_event["event_count"] == 0].index)
print(len(secondstg_target_key), len(not_target_key))

5369 2334


In [17]:
target_df = oof_df[oof_df["series_date_key"].isin(secondstg_target_key)].reset_index(drop=True)
display(target_df)

,series_id,step,anglez,enmo,event,event_onset,event_wakeup,date,series_date_key,series_date_key_str,fold,class_pred
0,0ec9fc461819,0,-11.080400,0.0335,0.0,0,0,2019-03-27,400,0ec9fc461819_2019-03-27,0,0.053497
1,0ec9fc461819,1,-16.320299,0.0562,0.0,0,0,2019-03-27,400,0ec9fc461819_2019-03-27,0,0.014214
2,0ec9fc461819,2,-18.830999,0.0441,0.0,0,0,2019-03-27,400,0ec9fc461819_2019-03-27,0,0.007229
3,0ec9fc461819,3,-18.533800,0.0408,0.0,0,0,2019-03-27,400,0ec9fc461819_2019-03-27,0,0.006878
4,0ec9fc461819,4,-11.525700,0.0520,0.0,0,0,2019-03-27,400,0ec9fc461819_2019-03-27,0,0.005859
...,...,...,...,...,...,...,...,...,...,...,...,...
90338284,f981a0805fd0,78115,-11.604600,0.0034,0.0,0,0,2019-01-07,7505,f981a0805fd0_2019-01-07,4,-1.000000
90338285,f981a0805fd0,78116,-11.499600,0.0034,0.0,0,0,2019-01-07,7505,f981a0805fd0_2019-01-07,4,-1.000000
90338286,f981a0805fd0,78117,-11.658200,0.0034,0.0,0,0,2019-01-07,7505,f981a0805fd0_2019-01-07,4,-1.000000
90338287,f981a0805fd0,78118,-11.459900,0.0032,0.0,0,0,2019-01-07,7505,f981a0805fd0_2019-01-07,4,-1.000000


In [18]:
target_df.to_parquet(os.path.join(output_dir, exp_name, "oof_df_secondstg.parquet"))